In [10]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [11]:
import os, sys 
%cd ~/nbs/
sys.path.append(os.path.join(os.getcwd(), "orig" ))

%matplotlib inline 
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

/home/ubuntu/nbs


1. Create batches of validation 
2. Create batches of training 
3. Use average of different models and fit it collectively 
4. Fit the batches
5. Run the training on the train and validation data. 
6. Predict on the test 
7. Run prediction on validation set 
8. See what of the predicted on the validation set is correct 
9. Submit tests to backend 

In [12]:
path = 'data/dogscats/'
model_path = path + 'model/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size = 16

In [13]:
training_batches = get_batches(path + 'train', shuffle=False, batch_size=batch_size)
validation_batches = get_batches(path + 'valid', shuffle=False, batch_size=batch_size)


Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [14]:
??get_classes
(val_classes, training_classes, val_labels, training_labels, 
val_filenames, filenames, test_filenames) = get_classes(path)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


# Create an ensemble of different models

1. Train model with last layer all other layer traininable is false
2. Add model with everything but the last layer. This creates an ensemble. 
3. Fine tune just the dense layers of this model. 
4. Add data augmentation, fine tuning dense layers without computation

In [15]:
model = Vgg16().model
# print(model.summary())
conv_layers, fc_layers = split_at(model, Convolution2D)


In [ ]:
conv_model = Sequential(conv_layers)
# print(conv_model.summary())

print(validation_batches.nb_sample)

2000


In [ ]:
val_predictions = conv_model.predict_generator(validation_batches, validation_batches.nb_sample)
training_predictions = conv_model.predict_generator(training_batches, training_batches.nb_sample)
save_array(model_path + 'valconvlayer_predictions.bc', val_predictions)
save_array(model_path + 'trainconvlayer_predictions.bc', training_predictions)





In [ ]:
validation = get_data(path+'valid')
training = get_data(path+'train')
save_array(model_path + 'valid_data.bc', validation)
save_array(model_path + 'training_data.bc', training)

In [ ]:
model.pop()
model.pop()


In [ ]:
# This is the prediction without the last two layers
# which is dropout and dense 
ll_val_predictions = model.predict_generator(validation_batches, validation_batches.nb_sample)
ll_trn_predictions = model.predict_generator(training_batches, training_batches.nb_sample)

save_array(model_path + 'model_val.bc', model_val_predictions)
save_array(model_path + 'model_trn.bc', model_trn_predictions)

In [ ]:
test = get_data(path+'test')
save_array(model_path + 'test_data.bc', test)



In [ ]:
test = load_array(model_path + 'test_data.bc')

## Last layer

In [ ]:
# This is the last last layer. The original model 
# had dropout, and dense but not BatchNormalization
# This is simply adding BatchNormalization in 
def get_last_layer():
    lastLayer = [BatchNormalization(input_shape=(4096,)), 
                Dropout(0.5),
                Dense(2,activation='softmax')]
    return lastLayer
    

In [ ]:
def train_last_layer():
    lastLayers = get_last_layer()
    ll_model = Sequential(lastLayers)
    ll_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    ll_model.optimizer.lr = 1e-5
    ll_model.fit(ll_trn_predictions, training_labels, validation_data=(ll_val_predictions, validation_labels), nb_epoch=12)
    ll_model.optimizer.lr=1e-7
    ll_model.fit(ll_trn_predictions, training_labels, validation_data=(ll_val_predictions, validation_labels), nb_epoch=1)
    ll_model.save_weights(model_path+'ll_bn' + i + '.h5')
    
    vgg = VGG16()
    model = vgg.model
    model.pop(); model.pop(); model.pop()
    for layer in model.layers(): 
        layer.trainable = False 
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    ll_layers = get_last_layer()
    for layer in ll_layers: 
        model.add(layer)

    for layer1, layer2 in zip(ll_model.layers, model.layers[-3:]): 
        layer2.set_weights(ll.get_weights())
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuacy'])
    model.save_weights(model_path + 'bn' + i + '.h5')
    return model 
    

## Dense model